You're right to focus on Custom Named Entity Recognition (NER) for extracting performance metrics and values, as it's one of the most robust and scalable solutions for this problem, especially when dealing with varied or complex sentence structures.

Let's break down the steps in detail, with a practical example using `spaCy`, a popular and powerful NLP library in Python.

**Goal:** Train a custom NER model to identify "PERFORMANCE\_METRIC" and "VALUE" entities in sentences like:

  * "The **CPU utilization** was **75%**."
  * "**Latency** improved to **10 ms**."
  * "**Throughput** reached **1.2 Gbps**."

-----

### **Steps for Custom Named Entity Recognition (NER)**

#### **Step 1: Data Annotation (Labeling)**

This is the most crucial and often the most time-consuming step. You need to manually label examples of your text with the custom entities you want to extract.

**Idea:** For each sentence, identify the exact spans of text that correspond to a `PERFORMANCE_METRIC` and a `VALUE`.

**Format:** spaCy typically expects training data in a list of tuples, where each tuple contains:

1.  The text of the sentence.
2.  A dictionary containing a list of entities. Each entity is a tuple: `(start_index, end_index, 'ENTITY_TYPE')`.

**Example Data Annotation:**

Let's consider these sentences:

  * "The CPU utilization was 75%."
  * "Latency improved to 10 ms."
  * "Throughput reached 1.2 Gbps on average."
  * "We observed 99.9% uptime for the service."
  * "The network bandwidth is 100 Mbps."

Here's how you'd annotate them:

In [ ]:
TRAIN_DATA = [
    ("The CPU utilization was 75%.", {
        "entities": [(4, 19, "PERFORMANCE_METRIC"), (24, 28, "VALUE")]
    }),
    ("Latency improved to 10 ms.", {
        "entities": [(0, 7, "PERFORMANCE_METRIC"), (20, 25, "VALUE")]
    }),
    ("Throughput reached 1.2 Gbps on average.", {
        "entities": [(0, 10, "PERFORMANCE_METRIC"), (19, 27, "VALUE")]
    }),
    ("We observed 99.9% uptime for the service.", {
        "entities": [(12, 17, "VALUE"), (18, 23, "PERFORMANCE_METRIC")] # Notice order can vary
    }),
    ("The network bandwidth is 100 Mbps.", {
        "entities": [(4, 20, "PERFORMANCE_METRIC"), (24, 32, "VALUE")]
    }),
    # Add many more examples here... ideally hundreds or thousands for good performance
    ("Memory usage peaked at 80%.", {
        "entities": [(0, 12, "PERFORMANCE_METRIC"), (22, 26, "VALUE")]
    }),
    ("The disk I/O was 150 MB/s.", {
        "entities": [(4, 13, "PERFORMANCE_METRIC"), (18, 25, "VALUE")]
    }),
    ("Response time increased to 300ms.", {
        "entities": [(0, 12, "PERFORMANCE_METRIC"), (24, 30, "VALUE")]
    }),
    ("Page load speed is 2.5 seconds.", {
        "entities": [(0, 15, "PERFORMANCE_METRIC"), (19, 30, "VALUE")]
    }),
    ("Error rate dropped to 0.1%.", {
        "entities": [(0, 10, "PERFORMANCE_METRIC"), (21, 26, "VALUE")]
    })
]

**Tips for Annotation:**

  * **Consistency:** Be absolutely consistent with your labels. If "CPU utilization" is a `PERFORMANCE_METRIC` once, it must always be.
  * **Exact Spans:** Ensure the start and end indices perfectly cover the entity, without extra spaces or partial words.
  * **Ambiguity:** If a phrase can sometimes be a metric and sometimes not, you might need more complex rules or more diverse training data.
  * **Diversity:** Include a wide variety of sentence structures, phrasing, and metrics/values. Don't just use simple "X is Y" sentences. Include sentences with conjunctions, different verbs ("reached", "improved", "peaked at", "dropped to"), and varied unit formats.
  * **Quantity:** For a production-ready model, you'd typically need hundreds to thousands of annotated examples. The more varied and representative your data, the better your model will perform.

#### **Step 2: Initialize and Configure the spaCy Model**

You'll start with a blank spaCy model and add the NER component to it.

In [ ]:
import spacy
from spacy.training import Example
import random

# 1. Create a blank spaCy model
# You can specify the language, e.g., "en" for English
nlp = spacy.blank("en")

# 2. Add the 'ner' pipeline component
# This creates an empty NER component if it doesn't exist.
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

# 3. Add custom labels to the NER component
ner.add_label("PERFORMANCE_METRIC")
ner.add_label("VALUE")

print("Labels added to NER component:", ner.labels)

#### **Step 3: Prepare the Training Loop**

You'll iterate multiple times over your training data. Each iteration is called an "epoch." During each epoch, the model adjusts its internal weights based on the annotated examples.

In [ ]:
# Create a list of Example objects from your TRAIN_DATA
examples = []
for text, annotations in TRAIN_DATA:
    examples.append(Example.from_dict(nlp.make_doc(text), annotations))

# Get the names of the pipeline components to disable during training
# We only want to train the NER component, not others like tagger or parser
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"] # 'trf_' components are for transformer models
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Training parameters
n_iter = 50 # Number of training iterations (epochs)
dropout = 0.5 # Dropout rate to prevent overfitting

#### **Step 4: Train the Model**

This is where the actual learning happens.

In [ ]:
# Start the training
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    print("Training the model...")
    for itn in range(n_iter):
        random.shuffle(examples) # Shuffle data for better training
        losses = {}
        for example in examples:
            nlp.update(
                [example],  # batch of examples
                drop=dropout, # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses,
            )
        print(f"Epoch {itn+1}/{n_iter} - Losses: {losses}")

print("Training complete!")

#### **Step 5: Test the Model**

After training, you can use the model to predict entities on new, unseen sentences.

In [ ]:
# Test sentences
test_sentences = [
    "Our system's uptime is 99.99%.",
    "The new server reduced response time to 50ms.",
    "Data transfer rate peaked at 1.5 Gbps.",
    "We need to monitor CPU usage which is currently 60% and memory consumption at 70%.",
    "What is the network latency?", # Should ideally not find values if no value
    "The average temperature is 25 degrees Celsius." # Should not tag as performance metric/value
]

print("\n--- Testing the trained model ---")
for text in test_sentences:
    doc = nlp(text)
    print(f"\nSentence: {doc.text}")
    for ent in doc.ents:
        print(f"  - Entity: '{ent.text}' | Label: '{ent.label_}'")

**Expected Output (Example, actual output might vary slightly based on training):**

```
--- Testing the trained model ---

Sentence: Our system's uptime is 99.99%.
  - Entity: 'uptime' | Label: 'PERFORMANCE_METRIC'
  - Entity: '99.99%' | Label: 'VALUE'

Sentence: The new server reduced response time to 50ms.
  - Entity: 'response time' | Label: 'PERFORMANCE_METRIC'
  - Entity: '50ms' | Label: 'VALUE'

Sentence: Data transfer rate peaked at 1.5 Gbps.
  - Entity: 'Data transfer rate' | Label: 'PERFORMANCE_METRIC'
  - Entity: '1.5 Gbps' | Label: 'VALUE'

Sentence: We need to monitor CPU usage which is currently 60% and memory consumption at 70%.
  - Entity: 'CPU usage' | Label: 'PERFORMANCE_METRIC'
  - Entity: '60%' | Label: 'VALUE'
  - Entity: 'memory consumption' | Label: 'PERFORMANCE_METRIC'
  - Entity: '70%' | Label: 'VALUE'

Sentence: What is the network latency?
  - Entity: 'network latency' | Label: 'PERFORMANCE_METRIC' # Model might pick this up based on training, even without a value

Sentence: The average temperature is 25 degrees Celsius.
 # (Ideally, no entities should be found for this, as it's not a performance metric)
```

#### **Step 6: Save and Load the Model (for production use)**

Once trained, you'll want to save your model so you don't have to retrain it every time.

In [ ]:
# Save the model
output_dir = "./custom_ner_model"
nlp.to_disk(output_dir)
print(f"\nModel saved to {output_dir}")

# Load the model later
print("\n--- Loading the saved model ---")
loaded_nlp = spacy.load(output_dir)
doc = loaded_nlp("The network bandwidth is 200 Mbps now.")
print(f"Sentence: {doc.text}")
for ent in doc.ents:
    print(f"  - Entity: '{ent.text}' | Label: '{ent.label_}'")

-----

### **Key Considerations and Best Practices for Custom NER:**

1.  **Data Quality and Quantity:**

      * **Garbage In, Garbage Out:** The performance of your NER model is directly proportional to the quality and quantity of your annotated data.
      * **More Data is Better:** Aim for hundreds, ideally thousands, of diverse examples.
      * **Balance:** Ensure your training data has a balanced representation of all entity types you want to recognize.
      * **Real-World Data:** Annotate data that is representative of the text your model will encounter in production.

2.  **Annotation Tools:**

      * For larger datasets, manual annotation becomes tedious. Consider using annotation tools like:
          * **Prodigy (by spaCy):** A highly efficient annotation tool specifically designed for spaCy.
          * **Doccano:** An open-source annotation tool for various NLP tasks.
          * **Label Studio:** Another versatile open-source tool.

3.  **Preprocessing:**

      * While spaCy handles much of it, ensure your input text is clean (e.g., consistent casing if that matters, handling special characters).

4.  **Model Architecture (Advanced):**

      * `spaCy`'s `ner` component is based on a statistical model (often a neural network like a multi-layer perceptron or more recently, transformer-based).
      * For even higher performance, especially with limited data, you might consider fine-tuning pre-trained language models (like BERT, RoBERTa, etc.) using `Hugging Face Transformers` or `spaCy`'s integration with Transformers. This would involve a different training setup but leverages the massive knowledge embedded in these large models.

5.  **Evaluation:**

      * Always split your annotated data into training, validation, and test sets.
      * Train on the training set, use the validation set to tune hyperparameters, and *only* evaluate final performance on the unseen test set.
      * Metrics: Precision, Recall, and F1-score are standard for NER evaluation.

6.  **Iterative Improvement:**

      * NER development is often iterative. Train a first model, analyze its errors, annotate more data based on common error patterns, and retrain.

By following these detailed steps and best practices, you can successfully build a robust custom NER model to extract performance metrics and values from your text data.